# Extended Kalman Filter

Use uncertain information (sensor data) and knowledge of how the system behaves (educated guess) to understand the state of a system which is continuously changing

## The State Estimate

$$ x_k = \begin{bmatrix} \theta \\ \dot{\theta} \\ \ddot{\theta} \end{bmatrix} $$

where $\theta$ is the absolute angle to the object

## The Covariance Matrix

$$ P = \begin{bmatrix} \Sigma_{pp} & \Sigma_{pv} & \Sigma_{pa} \\ 
                       \Sigma_{vp} & \Sigma_{vv} & \Sigma_{va} \\
                       \Sigma_{ap} & \Sigma_{av} & \Sigma_{aa} \end{bmatrix}$$

where $ \Sigma_{ij} $ is the degree of variance between the _i'th_ and _j'th_ state variable.

## The Prediction Matrix, Control Matrix and Control Vector

Indicates how we move from one state to the next

$$ \begin{split}
   p_k &= p_{k-1} + \Delta t \cdot v_{k-1} \\
   v_k &= v_{k-1} + \Delta t \cdot a_{k-1} \\
   a_k &= \beta a_{k-1}
   \end{split}
$$

where $\beta$ represents constant acceleration ($\beta = 1$), no acceleration ($\beta = 0$) or a decaying acceleration ($0 \lt \beta \lt 1$)

Thus,

$$ \begin{split} \hat{x}_k &= \begin{bmatrix} 1 & \Delta t & 0 \\
                                              0 & 1 & \Delta t \\
                                              0 & 0 & \beta \end{bmatrix}\hat{x}_{k-1} \\ \\
                           &= F_k\ \hat{x}_{k-1} \end{split} $$

where $F_k$ is the prediction matrix

## Factoring in Uncertainty

Eg. uncertainty from modelling, noise, etc

Update the prediction matrix as follows,

$$ P_k = F_k\ P_{k-1}\ F_k^T + Q_k$$

In other words, the new uncertainty is predicted from the old uncertainty, with some additional uncertainty from the environment

## The Sensor Uncertainty Matrix

$$ \mu_{expected} = H_k\ \hat{x}_k $$
$$ \Sigma_{expected} = H_k\ P_k\ H_k^T $$
$$ y = z - h(x) $$

where $H_k$ is a matrix which models the sensors

In [ ]:
import numpy as np, time

In [ ]:
class KalmanFilter():
    """
    Inputs:
        Ts      Sampling time of the model (used in `predict()` method). Can be updated.
        Q, R    Uncertainty matrices.
                Higher Q = more uncertainty in model.
                Higher R = more uncertainty in sensor = slower response to measurements
        a       Acceleration change from one instant to the next. `a = 0` for no acceleration, or `a = 1` for constant
        
    Keeps track of:
        x       the state estimate, θ_pos, θ_vel and θ_acc. Measured in rads
        P       the covariance matrix
    """
    def __init__(self, Ts, Q, R, a=1):
        self.Ts = Ts
        self.x = np.matrix([[0],  # position
                            [0],  # velocity
                            [0]]) # acceleration
        self.P = np.matrix(np.eye(3))
        self.F = np.matrix([[1, Ts,  0],
                            [0,  1, Ts],
                            [0,  0,  a]])
        self.Q = np.matrix([[Q*(Ts**2),    0, 0],
                            [        0, Q*Ts, 0],
                            [        0,    0, Q]])
        self.R = np.rad2deg(R)
        
    def predict(self):
        self.x = self.F * self.x
        self.P = (self.F * self.P * self.F.T) + self.Q

    def update(self, z): # z in degrees
        y = z - self.h(self.x)
        H = self.h_dash(self.x)
        
        K = (self.P * H.T) / (H * self.P * H.T + self.R)
        self.x = self.x  +  K * y
        self.P = (np.eye(3) - K * H) * self.P
    
    def h(self, x): return np.rad2deg(x.item(0))
    def h_dash(self, x): return np.matrix([np.rad2deg(1), 0, 0])

    def get_pos(self): return np.rad2deg(self.x.item(0))
    def get_vel(self): return np.rad2deg(self.x.item(1))
    def get_acc(self): return np.rad2deg(self.x.item(2))

    def predict_ahead(self, T):
        """ predict T seconds ahead using the model, F """
        F = np.matrix(np.copy(self.F))

        F[0,1], F[1,2] = T, T
        
        return np.rad2deg((F*self.x).item(0))
    
    def update_T(self, ΔT):
        self.Ts = ΔT
        self.F[0,1], self.F[1,2] = ΔT, ΔT
        self.Q[0,0] = self.Q[2,2]*ΔT**2 / 2
        self.Q[1,1] = self.Q[2,2]*ΔT
    
    # methods which assume that the time between updates from the nn (ΔT) isn't constant  approach
    def start_timer(self): self.t = time.time()
    def better_update(self, z):
        # first call the predict function using the actual passed time as T
        ΔT = time.time() - self.t
        self.t = time.time()
        self.update_T(ΔT)
        self.predict() # update x using the time since last update
        
        # next, perform the regular update stuff
        self.update(z)
    def get_cur_est_pos(self):
        return self.predict_ahead(time.time() - self.t)

In [ ]:
class ExtendedKalmanFilter(KalmanFilter):
    """
    Inputs:
        Ts                  Sampling time of the model (used in `predict()` method). Can be updated.
        Q, R                Uncertainty matrices.
                            Higher Q = more uncertainty in model.
                            Higher R = more uncertainty in sensor = slower response to measurements
        camera_FOV_deg      The angular width of the camera. picam width, height = 62.2ᵒ, 48.8ᵒ for max FOV
        
    Keeps track of:
        x       the state estimate, θ_pos, θ_vel and θ_acc. Measured in rads
        P       the covariance matrix
    """
    def __init__(self, Ts, Q, R, a=1, camera_FOV_deg=62.2):
        super().__init__(Ts, Q=Q, R=R, a=a)  # calling init method in KalmanFilter
        self.camera_FOV_rad = np.deg2rad(camera_FOV_deg)

    def update(self, z): # z in degrees, gets converted to pixels. Done this way so that the KF and EKF both take in angles
        y = self.angle_to_pixel(z) - self.h(self.x)
        H = self.h_dash(self.x)
        
        K = (self.P * H.T) / (H * self.P * H.T + self.R)
        self.x = self.x  +  K * y
        self.P = (np.eye(3) - K * H) * self.P

    def h(self, x):  # overriding method in KalmanFilter
        """ convert x, an angle in rads, into a pixel """
        return np.tan(x.item(0)) / np.tan(self.camera_FOV_rad/2)
    
    def angle_to_pixel(self, x):
        """ convert x, an angle in degrees, into a pixel """
        return np.tan(np.deg2rad(x)) / np.tan(self.camera_FOV_rad/2)
    
    def h_dash(self, x):  # overriding method in KalmanFilter
        _h_dash = (1 + np.power(np.tan(x.item(0)),2)) / np.tan(self.camera_FOV_rad/2)
        return np.matrix([_h_dash, 0, 0])

# Debugging: simulated data

In [ ]:
if __name__ == '__main__':
    Ts   = 0.050
    T_nn = 0.200
    sim_time = 15
    nn_delay = 1  # nn_delay measured in units of Ts
    FOV_deg  = 62.2  # degrees

    def noise(scale): return (np.random.random() - 0.5)*scale

    def ref_unit_sigmoid(t, sim_time): return 2/(1 + 3**(-t + sim_time/2)) - 1
    def ref_unit_step(t, sim_time): return 1
    def ref_unit_sin(t, sim_time): return np.sin(t*0.8)
    
    def pixel_to_angle(norm_pixel, cam_fov_rad):
        return np.rad2deg(np.arctan(norm_pixel * np.tan(cam_fov_rad/2)))

    nn_input_funcs = [ref_unit_sigmoid, ref_unit_step, ref_unit_sin]

In [ ]:
if __name__ == '__main__':
    def derivative(f):
        df = [(x - y)/Ts for x, y in zip(f[1:], f[0:-1])]
        df.append(df[-1]) # make sure df has the same length as f
        return df

    def plot_simulation():
        plt.subplot(311)
        plt.plot(t_arr, nn_input_arr, label='input')
        plt.plot(t_arr, KF_pos, label='KF angle position')
        plt.plot(t_arr, EKF_pos, label='EKF angle position')
        plt.legend(loc='lower right'); plt.grid()

        plt.subplot(312)
        plt.plot(t_arr, derivative(nn_input_arr), label='input velocity')
        plt.plot(t_arr, KF_vel, label='KF angle velocity')
        plt.plot(t_arr, EKF_vel, label='EKF angle velocity')
        plt.legend(loc='lower right'); plt.grid()

        plt.subplot(313)
        plt.plot(t_arr, derivative(derivative(nn_input_arr)), label='input acceleration')
        plt.plot(t_arr, KF_acc, label='KF angle acceleration')
        plt.plot(t_arr, EKF_acc, label='EKF angle acceleration')
        plt.legend(loc='lower right'); plt.grid()

        fig = plt.gcf(); fig.set_size_inches(18.5, 6, forward=True)

        plt.show()

In [ ]:
if __name__ == '__main__':
    import matplotlib.pyplot as plt, utils
    
    for nn_input_func in nn_input_funcs:
        KF  =         KalmanFilter(Ts=T_nn, Q=0.5, R=0.05, a=0.94)
        EKF = ExtendedKalmanFilter(Ts=T_nn, Q=0.5, R=0.05, a=0.94, camera_FOV_deg=FOV_deg)
        
        KF_pos, KF_vel, KF_acc = [], [], []
        EKF_pos, EKF_vel, EKF_acc = [], [], []
        nn_input_arr, t_arr = [], []
        
        nn_input_delayed = np.zeros(nn_delay)
        
        for t in np.arange(0, sim_time, Ts):
            nn_input = nn_input_func(t, sim_time)  # get input
            nn_input_delayed[0] = nn_input  # put at front of buffer
            nn_input_delayed = np.roll(nn_input_delayed, 1)  # roll buffer

            if (t % T_nn < 0.01) or -0.01 < abs(t % T_nn) - T_nn < 0.01:
                actual_measurement = pixel_to_angle(nn_input_delayed[-1] + noise(0.1), np.deg2rad(FOV_deg))
                KF.predict();
                KF.update(actual_measurement)
                EKF.predict();
                EKF.update(actual_measurement)
                
                KF_pos.append(KF.get_pos()); EKF_pos.append(EKF.get_pos())
                
                t_no_update = 0
            else:
                t_no_update += Ts
                KF_pos.append(KF.predict_ahead(t_no_update))
                EKF_pos.append(EKF.predict_ahead(t_no_update))

            nn_input_arr.append(pixel_to_angle(nn_input, np.deg2rad(FOV_deg)))

            KF_vel.append(KF.get_vel()); EKF_vel.append(EKF.get_vel())
            KF_acc.append(KF.get_acc()); EKF_acc.append(EKF.get_acc())
            t_arr.append(t)
            
        plot_simulation()

# Test the `KF` / `EKF` in real time

In [ ]:
if __name__ == '__main__':
    import time
    
    EKF = ExtendedKalmanFilter(Ts=0.1, Q=5, R=10)

    pos_arr = []
    nn_arr = []
    nn_with_noise_arr = []
    vel_arr = []
    t_arr = []
    
    EKF.start_timer()
    for i in range(100):
        # hugely variable amount of time between updates from nn --> t in [0.75, 0.325]
        t = 0.2 + ((np.random.random() - 0.5)/4)
        t_arr.append(t)
        time.sleep(t)
        
        # update EKF with sinusoidal shape + noise
        nn = 10*np.sin(i/10)
        nn_with_noise = nn + (np.random.random() - 0.5)*4
        EKF.better_update(nn_with_noise)
        
        # log data
        nn_arr.append(nn)
        nn_with_noise_arr.append(nn_with_noise)
        pos_arr.append(EKF.get_pos())
        vel_arr.append(EKF.get_vel())

    plt.plot(np.cumsum(t_arr), pos_arr, label='EKF estimated position')#; plt.ylim(0, 150)
    plt.plot(np.cumsum(t_arr), nn_arr, label='Actual position')
    plt.plot(np.cumsum(t_arr), nn_with_noise_arr, label='Sensed position (actual position + noise)')
    utils.plot_util(plt)

    plt.plot(np.cumsum(t_arr), vel_arr, label='velocity')
    utils.plot_util(plt)

    plt.stem(np.cumsum(t_arr), t_arr, label='time')
    utils.plot_util(plt)

In [ ]:
if __name__ == '__main__':
    EKF = ExtendedKalmanFilter(Ts=0.1, Q=5, R=10)
    EKF.start_timer()

    t = time.time()
    n = 1000
    for i in range(n):
        EKF.better_update(np.random.random())
    print('testing speed of EKF: %d loops in %dms' % (n, (time.time() - t)*1000))

# Debugging: real data

## Load the pre-recorded data

In [ ]:
if __name__ == '__main__':
    import pandas as pd
    
    df_loaded = pd.read_csv('logged_data_3.csv', index_col=0)
    # print('Columns:', [i for i in df_loaded.columns])

    phi_yaw_arr = df_loaded.iloc[:,0].values
    phi_pitch_arr = df_loaded.iloc[:,1].values
    gc_yaw_arr = df_loaded.iloc[:,2].values
    gc_pitch_arr = df_loaded.iloc[:,3].values
    time_arr = df_loaded.iloc[:,4].values

## Show the size and timing of updates from the neural network

In [ ]:
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    from utils import plot_util

    nn_arr = []
    nn_t_arr = []
    prev_yaw = 0
    for i, yaw in enumerate(phi_yaw_arr):
        if yaw != prev_yaw:
            nn_arr.append(yaw)
            nn_t_arr.append(time_arr[i])
            prev_yaw = yaw

    plt.stem(nn_t_arr, nn_arr)
    plt.xlabel('Time [s]'); plt.ylabel('Angle estimate from NN [deg]')
    plot_util(plt)

## Simulate and plot a KF/EKF with an adjustable widget for easy modification of `Q` and `R`, using real data

In [ ]:
if __name__ == '__main__':
    def sim_and_plot(model, Q, R, a):
        if model == 'EKF':
            EKF = ExtendedKalmanFilter(Ts=0.05, Q=Q, R=R, a=a, camera_FOV_deg=62.2)
        else:
            EKF = KalmanFilter(Ts=0.05, Q=Q, R=R, a=a)

        results = []
        prev_t, prev_measurement = 0, 0

        for curr_t, measurement in zip(time_arr, phi_yaw_arr):
            if measurement != prev_measurement:
                EKF.update_T(curr_t - prev_t)
                EKF.predict()
                EKF.update(measurement)
                results.append(EKF.get_pos())
                prev_t, prev_measurement = curr_t, measurement
            else:
                results.append(EKF.predict_ahead(curr_t - prev_t))

        plt.stem(nn_t_arr, nn_arr, label='Raw NN estimate [deg]')
        plt.plot(time_arr, results, label='EKF estimate [deg]')
        plt.ylim(bottom=-30, top=35)
        plot_util(plt)

In [ ]:
if __name__ == '__main__':
    from ipywidgets import FloatSlider, Layout, interact
    # good values for KF: Q = 0.5, R = 0.005, a = 0.94
    #  ------------- EKF: Q =   5, R = 0.005, a = 0.95
    # interact(sim_and_plot, model=['KF', 'EKF'], Q=[5,0.5,0.05], R=[0.5,0.05,0.005], a=[0,0.95,1]);
    # Q = 10, R = 0.05, 

    # range for KF
    Q_slider = FloatSlider(min=0.05, max=10, step=0.05, layout=Layout(width='95%'))
    R_slider = FloatSlider(min=1, max=10, step=1, layout=Layout(width='95%'))
    
    # range for EKF:
#     Q_slider = FloatSlider(min=1, max=10, step=0.05, layout=Layout(width='95%'))
#     R_slider = FloatSlider(min=0.01, max=1, step=0.01, layout=Layout(width='95%'))
    
    
    interact(sim_and_plot, model=['KF', 'EKF'], Q=Q_slider, R=R_slider, a=[0,0.5,0.95,1]);